<a href="https://colab.research.google.com/github/brodyandre/etl_em_dados_textuais_json_com_regex/blob/main/etl_em_dados_textuais_json_com_regex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Documentação do Processo de Limpeza e Transformação de Dados de Locação de Imóveis

Objetivo

Este notebook tem como objetivo realizar o processo de limpeza, normalização e transformação de dados textuais extraídos de um arquivo JSON contendo informações sobre locações de imóveis. As etapas incluem manipulação de colunas, conversão de tipos de dados, formatação de valores monetários e ajustes em strings utilizando expressões regulares (Regex).


In [24]:
!pip install pandas

##Bibliotecas Necessárias

In [25]:
import numpy as np
import json
import pandas as pd
import re
import unicodedata

* pandas: Manipulação de DataFrames.

* numpy: Operações matemáticas otimizadas.

* json: Leitura de arquivos JSON.

* re: Operações com expressões regulares.

* unicodedata: Tratamento de caracteres especiais.

###1️⃣ Leitura dos Dados

In [26]:
with open('/content/dados_locacao_imoveis.json', 'r') as file:
    dados = json.load(file)

dados = pd.DataFrame(dados)
dados.head()

,dados_locacao
0,"{'apartamento': 'A101 (blocoAP)', 'datas_combi..."
1,"{'apartamento': 'A102 (blocoAP)', 'datas_combi..."
2,"{'apartamento': 'B201 (blocoAP)', 'datas_combi..."
3,"{'apartamento': 'B202 (blocoAP)', 'datas_combi..."
4,"{'apartamento': 'C301 (blocoAP)', 'datas_combi..."


* Abrimos o arquivo JSON e carregamos os dados em um DataFrame.

* Exibimos as primeiras linhas para visualização inicial.

###2️⃣ Normalização da Estrutura de Dados

In [27]:
dados = pd.json_normalize(dados['dados_locacao'])
dados

,apartamento,datas_combinadas_pagamento,datas_de_pagamento,valor_aluguel
0,A101 (blocoAP),"[01/06/2022, 01/07/2022]","[05/06/2022, 03/07/2022]","[$ 1000,0 reais, $ 2500,0 reais]"
1,A102 (blocoAP),"[02/06/2022, 02/07/2022]","[02/06/2022, 06/07/2022]","[$ 1100,0 reais, $ 2600,0 reais]"
2,B201 (blocoAP),"[03/06/2022, 03/07/2022]","[07/06/2022, 03/07/2022]","[$ 1200,0 reais, $ 2700,0 reais]"
3,B202 (blocoAP),"[04/06/2022, 04/07/2022]","[07/06/2022, 05/07/2022]","[$ 1300,0 reais, $ 2800,0 reais]"
4,C301 (blocoAP),"[05/06/2022, 05/07/2022]","[10/06/2022, 09/07/2022]","[$ 1400,0 reais, $ 2900,0 reais]"
5,C302 (blocoAP),"[06/06/2022, 06/07/2022]","[08/06/2022, 12/07/2022]","[$ 1500,0 reais, $ 1200,0 reais]"
6,D401 (blocoAP),"[07/06/2022, 07/07/2022]","[07/06/2022, 09/07/2022]","[$ 1600,0 reais, $ 1300,0 reais]"
7,D402 (blocoAP),"[08/06/2022, 08/07/2022]","[10/06/2022, 14/07/2022]","[$ 1700,0 reais, $ 1400,0 reais]"
8,E501 (blocoAP),"[09/06/2022, 09/07/2022]","[10/06/2022, 09/07/2022]","[$ 1800,0 reais, $ 1500,0 reais]"
9,E502 (blocoAP),"[10/06/2022, 10/07/2022]","[16/06/2022, 12/07/2022]","[$ 1900,0 reais, $ 1600,0 reais]"


* Utilizamos pd.json_normalize para transformar estruturas aninhadas em colunas planas, facilitando o tratamento dos dados.

###3️⃣ Explosão de Colunas

In [28]:
colunas = list(dados.columns)
dados = dados.explode(colunas[1:])
dados.reset_index(drop=True, inplace=True)
dados

,apartamento,datas_combinadas_pagamento,datas_de_pagamento,valor_aluguel
0,A101 (blocoAP),01/06/2022,05/06/2022,"$ 1000,0 reais"
1,A101 (blocoAP),01/07/2022,03/07/2022,"$ 2500,0 reais"
2,A102 (blocoAP),02/06/2022,02/06/2022,"$ 1100,0 reais"
3,A102 (blocoAP),02/07/2022,06/07/2022,"$ 2600,0 reais"
4,B201 (blocoAP),03/06/2022,07/06/2022,"$ 1200,0 reais"
5,B201 (blocoAP),03/07/2022,03/07/2022,"$ 2700,0 reais"
6,B202 (blocoAP),04/06/2022,07/06/2022,"$ 1300,0 reais"
7,B202 (blocoAP),04/07/2022,05/07/2022,"$ 2800,0 reais"
8,C301 (blocoAP),05/06/2022,10/06/2022,"$ 1400,0 reais"
9,C301 (blocoAP),05/07/2022,09/07/2022,"$ 2900,0 reais"


* Aplicamos o método .explode() para expandir listas que estão nas colunas.

* Reiniciamos os índices para um sequenciamento correto após a explosão.

###4️⃣ Remoção de Substrings com Regex

In [29]:
dados['apartamento'] = dados['apartamento'].str.replace(r'\(blocoAP\)', '', regex=True)
dados

,apartamento,datas_combinadas_pagamento,datas_de_pagamento,valor_aluguel
0,A101,01/06/2022,05/06/2022,"$ 1000,0 reais"
1,A101,01/07/2022,03/07/2022,"$ 2500,0 reais"
2,A102,02/06/2022,02/06/2022,"$ 1100,0 reais"
3,A102,02/07/2022,06/07/2022,"$ 2600,0 reais"
4,B201,03/06/2022,07/06/2022,"$ 1200,0 reais"
5,B201,03/07/2022,03/07/2022,"$ 2700,0 reais"
6,B202,04/06/2022,07/06/2022,"$ 1300,0 reais"
7,B202,04/07/2022,05/07/2022,"$ 2800,0 reais"
8,C301,05/06/2022,10/06/2022,"$ 1400,0 reais"
9,C301,05/07/2022,09/07/2022,"$ 2900,0 reais"


* Utilizamos Regex para remover a substring "(blocoAP)" de todas as linhas da coluna apartamento.

###5️⃣ Conversão de Datas para DateTime

In [30]:
dados['datas_combinadas_pagamento'] = pd.to_datetime(dados['datas_combinadas_pagamento'], dayfirst=True)
dados['datas_de_pagamento'] = pd.to_datetime(dados['datas_de_pagamento'], dayfirst=True)
dados.info()
dados

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   apartamento                 30 non-null     object        
 1   datas_combinadas_pagamento  30 non-null     datetime64[ns]
 2   datas_de_pagamento          30 non-null     datetime64[ns]
 3   valor_aluguel               30 non-null     object        
dtypes: datetime64[ns](2), object(2)
memory usage: 1.1+ KB


,apartamento,datas_combinadas_pagamento,datas_de_pagamento,valor_aluguel
0,A101,2022-06-01,2022-06-05,"$ 1000,0 reais"
1,A101,2022-07-01,2022-07-03,"$ 2500,0 reais"
2,A102,2022-06-02,2022-06-02,"$ 1100,0 reais"
3,A102,2022-07-02,2022-07-06,"$ 2600,0 reais"
4,B201,2022-06-03,2022-06-07,"$ 1200,0 reais"
5,B201,2022-07-03,2022-07-03,"$ 2700,0 reais"
6,B202,2022-06-04,2022-06-07,"$ 1300,0 reais"
7,B202,2022-07-04,2022-07-05,"$ 2800,0 reais"
8,C301,2022-06-05,2022-06-10,"$ 1400,0 reais"
9,C301,2022-07-05,2022-07-09,"$ 2900,0 reais"


* As colunas de datas são convertidas para o tipo datetime64[ns] para permitir operações temporais.

* O argumento dayfirst=True garante a interpretação correta no formato dia/mês/ano.

###6️⃣ Limpeza de Valores Monetários com Regex

In [31]:
dados['valor_aluguel'] = dados['valor_aluguel'].str.replace(r'[$\sreais]', '', regex=True)
dados

,apartamento,datas_combinadas_pagamento,datas_de_pagamento,valor_aluguel
0,A101,2022-06-01,2022-06-05,"1000,0"
1,A101,2022-07-01,2022-07-03,"2500,0"
2,A102,2022-06-02,2022-06-02,"1100,0"
3,A102,2022-07-02,2022-07-06,"2600,0"
4,B201,2022-06-03,2022-06-07,"1200,0"
5,B201,2022-07-03,2022-07-03,"2700,0"
6,B202,2022-06-04,2022-06-07,"1300,0"
7,B202,2022-07-04,2022-07-05,"2800,0"
8,C301,2022-06-05,2022-06-10,"1400,0"
9,C301,2022-07-05,2022-07-09,"2900,0"


* Utilizamos uma expressão regular para remover o símbolo $, espaços em branco e a palavra "reais" da coluna valor_aluguel.

###7️⃣ Substituição de Vírgula por Ponto nos Valores

In [32]:
dados['valor_aluguel'] = dados['valor_aluguel'].str.replace(',', '.', regex=True)
dados

,apartamento,datas_combinadas_pagamento,datas_de_pagamento,valor_aluguel
0,A101,2022-06-01,2022-06-05,1000.0
1,A101,2022-07-01,2022-07-03,2500.0
2,A102,2022-06-02,2022-06-02,1100.0
3,A102,2022-07-02,2022-07-06,2600.0
4,B201,2022-06-03,2022-06-07,1200.0
5,B201,2022-07-03,2022-07-03,2700.0
6,B202,2022-06-04,2022-06-07,1300.0
7,B202,2022-07-04,2022-07-05,2800.0
8,C301,2022-06-05,2022-06-10,1400.0
9,C301,2022-07-05,2022-07-09,2900.0


* Ajustamos os valores para o padrão numérico internacional, substituindo "," por "." para conversão futura.

###8️⃣ Conversão para Tipo Numérico (Float)

In [33]:
dados['valor_aluguel'] = dados['valor_aluguel'].astype('float64')
dados.info()
dados


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   apartamento                 30 non-null     object        
 1   datas_combinadas_pagamento  30 non-null     datetime64[ns]
 2   datas_de_pagamento          30 non-null     datetime64[ns]
 3   valor_aluguel               30 non-null     float64       
dtypes: datetime64[ns](2), float64(1), object(1)
memory usage: 1.1+ KB


,apartamento,datas_combinadas_pagamento,datas_de_pagamento,valor_aluguel
0,A101,2022-06-01,2022-06-05,1000.0
1,A101,2022-07-01,2022-07-03,2500.0
2,A102,2022-06-02,2022-06-02,1100.0
3,A102,2022-07-02,2022-07-06,2600.0
4,B201,2022-06-03,2022-06-07,1200.0
5,B201,2022-07-03,2022-07-03,2700.0
6,B202,2022-06-04,2022-06-07,1300.0
7,B202,2022-07-04,2022-07-05,2800.0
8,C301,2022-06-05,2022-06-10,1400.0
9,C301,2022-07-05,2022-07-09,2900.0


* Finalmente, convertemos a coluna para o tipo float64 para permitir cálculos matemáticos.

##Conclusão

* Ao final deste processo, os dados estão:

* Normalizados;

* Com colunas devidamente tratadas;

* Tipos de dados ajustados para operações futuras.

Pronto para a próxima etapa? Análise e visualização dos dados podem ser realizadas de forma otimizada e precisa.